ESCALAMIENTO DE LOS DATOS 

Una problemática puede ser que los features estén en escalas diferentes. Por ejemplo que uno se maneje entre 0 y 1, mientras que otro trabaja en decenas de miles. 

* Se puede dividir por la desviación estándar. 
* Hacer una resta del promedio de los valortes de las _features_ (centrarlo a cero)

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [29]:
X = pd.read_csv('../data/processed/X.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross', axis=1)
X = X.drop('domestic_gross', axis=1)
X = X.drop('gross', axis=1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [32]:
scaler.mean_

array([3.40986946e+07, 2.00230035e+03, 2.13193706e+00, 1.08509297e+02,
       1.06763840e+04, 4.22391116e+07, 6.45610786e+00])

In [33]:
scaler.scale_

array([4.21823021e+07, 1.17658366e+01, 7.60726458e-01, 2.27231071e+01,
       1.95034933e+04, 2.31878733e+08, 1.06161391e+00])

In [34]:
# Es aquí donde se adecúa la escala de los features. 
scaler.transform(X_train)

array([[-0.64241858, -1.21541312, -0.37061555, ..., -0.27658553,
        -0.15197216, -1.18320592],
       [ 6.30362243,  0.39943168,  0.28665093, ...,  1.93163426,
         1.11161936,  0.60652195],
       [-0.33423246, -0.02552748,  0.28665093, ..., -0.39974295,
        -0.09590837, -0.33544009],
       ...,
       [ 1.20669814,  0.82439084,  0.28665093, ...,  1.31056604,
         0.18441057,  0.32393334],
       [-0.73724508, -2.74526609,  0.28665093, ..., -0.48659919,
        -0.16922255, -0.33544009],
       [-0.45276558,  0.48442351, -0.37061555, ...,  1.15244052,
        -0.11747137, -0.24124388]])

In [35]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [36]:
from sklearn.linear_model import Lasso

model = Lasso()
model_scale = Lasso()

model.fit(X_train, y_train)
model_scale.fit(X_train_scaled, y_train)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.157e+17, tolerance: 9.902e+15
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [37]:
print(model.score(X_test,y_test))
print(model_scale.score(X_test_scaled,y_test))

0.524971485259376
0.5249714861435573


Hay algoritmos que son muy sensibles al escalonamiento de los datos. Sin embargo, hay una clase de algoritmos que NO, y ese algoritmo es la _regresión_. En cambio las _clasificaciones_ Sí.


<span style="font-size:larger;">**SIMPLIFICACIÓN DE TRANSFORMACIONES CON PIPELINES**</span>

Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar  sklearn.pipeline.make_pipeline que permite encadenar transformaciones a tus modelos. 

In [38]:
from sklearn.pipeline import make_pipeline

model_scale = make_pipeline(StandardScaler(),
                            Lasso())

model_scale.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('lasso', Lasso())])

In [39]:
print(model_scale.score(X_test,y_test))

0.5249714861435573


<span style="font-size:larger;">**CREACIÓN DE NUEVAS FEATURES DE FORMA AUTOMÁTICA**</span>


In [40]:
import numpy as np

In [41]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [42]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit(A)
transformer.transform(A)
# Para realizar las transformación más rápida se puede escribir:
# transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [43]:
X.shape

(4104, 7)

In [44]:
transformer = PolynomialFeatures(2) # Polinomio de grado 2
transformer.fit_transform(X).shape

(4104, 36)

In [45]:
model_poly = make_pipeline(PolynomialFeatures(2), Lasso())
model_poly.fit(X_train, y_train)
model.score(X_test,y_test)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.659e+19, tolerance: 9.902e+15
  model = cd_fast.enet_coordinate_descent(


0.524971485259376

<span style="font-size:larger;">**CREACIÓN DE FEATURES CATEGÓRICOS**</span>

* One-Hot Encoding: Toma todas las categorías y las trabaja como columnas, rellenando el DataFrame de 0 y 1; donde 1 significa que la característica está.  
* Encoding Binario: _Categoría_ --> _Número_ --> _Binario_ --> _Columnas_

In [46]:
movies_obj = pd.read_csv('../data/processed/movies_obj.csv')

In [47]:
movies_obj.shape

(5043, 11)

In [48]:
movies_obj.head()

,movie_title,color,language,country,genres,content_rating,plot_keywords,director_name,actor_1_name,actor_2_name,actor_3_name
0,Avatar,Color,English,USA,Action|Adventure|Fantasy|Sci-Fi,PG-13,avatar|future|marine|native|paraplegic,James Cameron,CCH Pounder,Joel David Moore,Wes Studi
1,Pirates of the Caribbean: At World's End,Color,English,USA,Action|Adventure|Fantasy,PG-13,goddess|marriage ceremony|marriage proposal|pi...,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport
2,Spectre,Color,English,UK,Action|Adventure|Thriller,PG-13,bomb|espionage|sequel|spy|terrorist,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman
3,The Dark Knight Rises,Color,English,USA,Action|Thriller,PG-13,deception|imprisonment|lawlessness|police offi...,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt
4,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN,NaN,Documentary,NaN,NaN,Doug Walker,Doug Walker,Rob Walker,NaN
